In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import print_function
import os
import sys
import cv2
import numpy as np
import pandas as pd
import time
import glob

import tensorflow as tf

# TensorFlow HUB

In [3]:
import tensorflow_hub as hub

sys.path.insert(0, '..')
from svs_reader import Slide

svspath = '/dev/shm/1304715.svs'
outfile = '../speed_profile_wsi.txt'

INCEPTION_V3 = 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'
NASNET = 'https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/1'
RESNET_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/1'
RESNET_V2_101 = 'https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/1'
RESNET_V2_152 = 'https://tfhub.dev/google/imagenet/resnet_v2_152/feature_vector/1'
MOBILENET_V2_035_128 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/1'
MOBILENET_V2_050_128 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_050_128/feature_vector/1'
MOBILENET_V2_100_128 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_128/feature_vector/1'
MOBILENET_V2_035_224 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/feature_vector/1'
MOBILENET_V2_050_224 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_050_224/feature_vector/1'
MOBILENET_V2_100_224 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/1'

module = hub.Module(MOBILENET_V2_035_224)
model_name = 'MobileNet-v2-035%'
height, width = hub.get_expected_image_size(module)
print(height, width)

process_mag = 20
process_size = height
batch_size = 32

image_in = tf.placeholder('float', [None, height, width, 3])
predict_op = module(image_in)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/Conv/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/Conv/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/Conv/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_variance:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/Conv/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/wei

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/project/weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_1/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_10/depthwise/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_10/depthwise/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_10/depthwise/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_10/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_10/depthwise/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_10/depthwise/BatchNorm/

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_12/depthwise/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_12/depthwise/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_12/depthwise/BatchNorm/moving_variance:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_12/depthwise/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_12/depthwise/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_12/depthwise/depthwise_weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_12/expand/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_14/depthwise/depthwise_weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_14/depthwise/depthwise_weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_14/expand/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_14/expand/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_14/expand/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_14/expand/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_14/expand/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_14/expand/Ba

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_16/expand/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_16/expand/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_16/expand/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_16/expand/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_16/expand/BatchNorm/moving_variance:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_16/expand/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_16/expand/weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_16

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_3/expand/weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_3/expand/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_3/project/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_3/project/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_3/project/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_3/project/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_3/project/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_3/project/BatchNorm/moving_mean
INFO:ten

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_5/project/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/BatchNorm/moving_variance:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_5/project/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/weights:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_5/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_6/depthwise/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_6/depthwi

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_8/depthwise/BatchNorm/beta:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_8/depthwise/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_8/depthwise/BatchNorm/gamma:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_8/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_8/depthwise/BatchNorm/moving_mean:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expanded_conv_8/depthwise/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_8/depthwise/BatchNorm/moving_variance:0 from checkpoint /tmp/tfhub_modules/25da67c244c5f4a57c79d6be4239c7d7ce141a3c/variables/variables with MobilenetV2/expand

# Set the constants - trained

In [4]:
# sys.path.insert(0, '..')
# from svs_reader import Slide  # our helpers for reading the svs format slide

# from densenet_small import Inference  # this is the model to profile
# svspath = '/dev/shm/TCGA-2A-A8VL-01Z-00-DX1.2C2BD6EF-EC17-4117-AE89-A22B67AFB233.svs'
# model_name = 'Densenet-S'

# outfile = '../speed_profile_wsi.txt'

# process_mag = 20
# process_size = 512
# batch_size = 4

# Initialize slide reading operations

In [5]:
# Initialize the slide, which is on a ramdisk for speed
preprocess_fn = lambda x: (x * (2/255.) - 1).astype(np.float32)
svs = Slide(slide_path    = svspath,
            preprocess_fn = preprocess_fn,
            process_mag   = process_mag,
            process_size  = process_size )
prefetch = min(len(svs.tile_list), 2048)
print(len(svs.tile_list))
def wrapped_fn(idx):
    coords = svs.tile_list[idx]
    img = svs._read_tile(coords)
    return img, idx
        
def read_region_at_index(idx):
    return tf.py_func(func = wrapped_fn,
                      inp  = [idx],
                      Tout = [tf.float32, tf.int64],
                      stateful = False)

# Using parallel preprocessing via tensorflow's Dataset API
ds = (tf.data.Dataset.from_generator(generator = svs.generate_index, output_types=tf.int64).
      map(read_region_at_index, num_parallel_calls=12).
      prefetch(prefetch).
      batch(batch_size))

iterator = ds.make_one_shot_iterator()
img_op, idx_op = iterator.get_next()
init_op = tf.global_variables_initializer()

48487


# Run trained model

In [6]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# with tf.Session(config=config) as sess:
#     sess.run(init_op)
#     model = Inference(sess=sess, x_dims=[process_size, process_size, 3])
    
#     ts = time.time()
#     while True:
#         try:
#             img, idx = sess.run([img_op, idx_op])
#             output = model.inference(img)
#         except tf.errors.OutOfRangeError:
#             print('Done')
#             dt = time.time() - ts
#             fps = float(len(svs.tile_list)) / dt
#             break
            

# outstr = '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
# model_name, svspath, process_mag, process_size, batch_size, len(svs.tile_list), dt, fps)
# print(outstr)
# with open(outfile, 'a+') as f:
#     f.write(outstr)

# Run TF-HUB Model

In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(init_op)
    
    ts = time.time()
    while True:
        try:
            img, idx = sess.run([img_op, idx_op])
            output = sess.run(predict_op, {image_in: img})
        except tf.errors.OutOfRangeError:
            print('Done')
            dt = time.time() - ts
            fps = float(len(svs.tile_list)) / dt
            break
            

outstr = '{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
model_name, svspath, process_mag, process_size, batch_size, len(svs.tile_list), dt, fps)
print(outstr)
with open(outfile, 'a+') as f:
    f.write(outstr)

Done
MobileNet-v2-035%	/dev/shm/1304715.svs	20	224	32	48487	238.684363127	203.142758767



In [8]:
!cat ../speed_profile_wsi.txt

MODEL	SLIDE	MAG	PROC_SIZE	BATCH	N_TILES	TOTAL_SECONDS	FPS
DenseNet	/dev/shm/1304715.svs	5	128	12	9173	42.0563509464	218.112123225
DenseNet	/dev/shm/1304715.svs	5	256	12	2262	22.7923920155	99.2436422849
DenseNet	/dev/shm/1304715.svs	10	256	12	9173	86.3586621284	106.21980209
DenseNet	/dev/shm/1304715.svs	20	512	2	9173	339.256626129	27.0385286344
UNet	/dev/shm/1304715.svs	5	128	12	9173	42.0985350609	217.893567715
UNet	/dev/shm/1304715.svs	5	256	12	2262	32.8275420666	68.905554836
UNet	/dev/shm/1304715.svs	10	256	12	9173	125.129557133	73.3080193856
UNet	/dev/shm/1304715.svs	20	512	2	9173	507.999552965	18.0571025042
FCN	/dev/shm/1304715.svs	5	128	12	9173	40.7403731346	225.157486155
FCN	/dev/shm/1304715.svs	5	256	12	2262	14.4017648697	157.064083497
FCN	/dev/shm/1304715.svs	10	256	12	9173	85.47696805	107.315458296
FCN	/dev/shm/1304715.svs	20	512	2	9173	457.348822117	20.0569008958
DenseNet-S	/dev/shm/1304715.svs	5	128	12	9173	39.8436200619	230.225064534
Densenet-S	/dev/shm/1304715.svs	5	256	2	2